# 2. Продвинутые операции

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Test") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

## 2.1 Оконные функции

Оконные функции (Window Functions) в PySpark позволяют выполнять вычисления над группами строк, сохраняя при этом индивидуальность каждой строки.

Основные компоненты оконных функций:
1. Оконная спецификация (WindowSpec) - определяет, какие строки будут включены в рамки окна для каждой строки
2. Оконная функция - функция, которая применяется к данным в рамках окна

**Ранжирование** (rank(), dense_rank(), row_number(), percent_rank())  
**Агрегатные функции** (sum(), avg(), max(), min())  
**Смещение** (lag(), lead())  
**Аналитические функции** (first(), last(), cume_dist(), ntile())  

In [2]:
# Задача: Найти разницу между текущей и предыдущей покупкой для каждого пользователя.

# Создаём DataFrame с покупками
sales = spark.createDataFrame([
    (1, "2023-01-10", 100),
    (1, "2023-01-15", 200),
    (2, "2023-01-12", 50),
    (1, "2023-01-20", 300)
], ["user_id", "date", "amount"])

# Определяем окно
from pyspark.sql.window import Window
window = Window.partitionBy("user_id").orderBy("date")

# Добавляем разницу с предыдущей покупкой
from pyspark.sql.functions import lag, col
sales_with_diff = sales.withColumn("prev_amount", lag("amount").over(window)).withColumn("diff", col("amount") - col("prev_amount"))
sales_with_diff.show()

+-------+----------+------+-----------+----+
|user_id|      date|amount|prev_amount|diff|
+-------+----------+------+-----------+----+
|      1|2023-01-10|   100|       NULL|NULL|
|      1|2023-01-15|   200|        100| 100|
|      1|2023-01-20|   300|        200| 100|
|      2|2023-01-12|    50|       NULL|NULL|
+-------+----------+------+-----------+----+



In [3]:
# Топ-3 товара по категориям

# Данные о товарах
products = spark.createDataFrame([
    (1, "Laptop", "Electronics", 999),
    (2, "Phone", "Electronics", 699),
    (3, "Desk", "Furniture", 200),
    (4, "Chair", "Furniture", 150)
], ["product_id", "name", "category", "price"])

# Окно для ранжирования
window = Window.partitionBy("category").orderBy(col("price").desc())

# Топ-3 в каждой категории
from pyspark.sql.functions import dense_rank
top_products = products.withColumn("rank", dense_rank().over(window)).filter(col("rank") <= 3)
top_products.show()

+----------+------+-----------+-----+----+
|product_id|  name|   category|price|rank|
+----------+------+-----------+-----+----+
|         1|Laptop|Electronics|  999|   1|
|         2| Phone|Electronics|  699|   2|
|         3|  Desk|  Furniture|  200|   1|
|         4| Chair|  Furniture|  150|   2|
+----------+------+-----------+-----+----+



## 2.2 Работа с датами и строками

## 2.3 Оптимизация (партиционирование, кэширование)

## 2.4 UDF (пользовательские функции)

## 2.5 Чтение/запись в разных форматах